In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [31]:
data=pd.read_csv("data2/production.csv")
categ= pd.get_dummies(data, columns=["Area"]) 
#categ.head()
data= pd.merge(data,categ,how="inner")
###

###droping columns
X=data.drop(["Area","Citrus Production(hg/ha)","Unnamed: 0","Unnamed: 0.1"],axis=1)
Y=data["Citrus Production(hg/ha)"]
mean_y=np.mean(Y)
std_var=np.var(Y)**(1/2)
print("Mean :"+str(mean_y))
print("VAR : "+str(std_var))
#normalizing target data
Y=(Y-mean_y)/std_var
X.shape

Mean :116805.98370435035
VAR : 203886.8023908932


(6873, 157)

In [32]:
X=X.loc[list(Y<3),:]
Y=Y.loc[list(Y<3)]
std_var=np.var(Y)**(1/2)*std_var
std_var

70772.44640851556

In [5]:
from sklearn.model_selection import train_test_split 

# Split into training and test set
X_train, X_test, y_train, y_test =train_test_split(X,Y, test_size = 0.2, random_state=17)

In [6]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import math
from sklearn.metrics import mean_squared_error as MSE


estimators =  [('xgb',xgb.XGBRegressor(n_estimators  = 100,learning_rate = 0.02, max_depth = 2)),
               ('lgbm',LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=12000, 
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.4, 
                                       ))]

ensemble = StackingRegressor(estimators      =  estimators,
                             final_estimator =  RandomForestRegressor())

# Fit ensemble using cross-validation
ensemble.fit(X_train.values, y_train)


[10:33:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:33:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:33:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:33:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:34:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:34:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


StackingRegressor(estimators=[('xgb',
                               XGBRegressor(learning_rate=0.02, max_depth=2)),
                              ('lgbm',
                               LGBMRegressor(bagging_fraction=0.75,
                                             bagging_freq=5, bagging_seed=7,
                                             feature_fraction=0.4,
                                             learning_rate=0.01, max_bin=200,
                                             n_estimators=12000, num_leaves=4,
                                             objective='regression'))],
                  final_estimator=RandomForestRegressor())

In [7]:
from sklearn.metrics import mean_squared_error
# Prediction
predictions = ensemble.predict(X_train.values)
print('Root Mean Square Error train = ' + str(math.sqrt(mean_squared_error(y_train, predictions))))
# Prediction
predictions = ensemble.predict(X_test.values)
print('Root Mean Square Error test = ' + str(math.sqrt(mean_squared_error(y_test, predictions))))

Root Mean Square Error train = 0.1484954421201869
Root Mean Square Error test = 0.17912609658900502


In [11]:
# Prediction
predictions = ensemble.predict(X_train.values)
print('Root Mean Square Error test = ' + str(math.sqrt(mean_squared_error(y_train, predictions))))

Root Mean Square Error test = 0.39740929250081297


In [8]:
from sklearn.ensemble import ExtraTreesRegressor

clf = ExtraTreesRegressor(n_estimators=1000, max_depth=8,min_samples_split=2, random_state=2)
# Fit rf to the training set    
clf.fit(X_train.values,y_train) 
# 
predict1 = clf.predict(X_test)
predict2 = clf.predict(X_train)
print('Root Mean Square Error test = ' + str(math.sqrt((MSE(y_test, predict1)))))
print('Root Mean Square Error train = ' + str(math.sqrt((MSE(y_train, predict2)))))


Root Mean Square Error test = 0.24188257322691975
Root Mean Square Error train = 0.23861329923405672


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(157,16)  # 6*6 from image dimension
        self.fc2 = nn.Linear(16,4)
        self.fc3 = nn.Linear(4,1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (fc1): Linear(in_features=157, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=4, bias=True)
  (fc3): Linear(in_features=4, out_features=1, bias=True)
)


In [12]:
Xt = torch.from_numpy(X_train.values)
Yt = torch.from_numpy(y_train.values)

In [13]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(Xt)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #2 'mat1' in call to _th_addmm

In [52]:
res=pd.read_csv("result.csv")
res

,algo,param,test mse(X),train mse(X),test mse(X reduced),train mse(X reduced)
0,Linear regression,NaN,0.826416,0.726966,0.172370,0.169471
1,Linear regression with CV,NaN,0.976110,0.700230,0.194807,0.037950
2,Elastic Net (tunning with gridsearch),{'lasso__alpha': 0.01},0.770946,0.643378,0.182603,0.179601
3,SGDRegressor,"{loss=squared_epsilon_insensitive,learning_rat...",0.770946,0.643378,0.175284,0.171893
4,SVR,"{kernel='rbf', C=10000, gamma='auto',coef0=1,v...",0.630053,0.092600,0.204866,0.078860
5,RF,{n_estimator=100},0.188616,0.145480,0.130241,0.045007
6,XGBoost (tunning with gridsearch),"{'gamma': 0.01, 'learning rate': 0.01, 'max_de...",0.357840,0.148457,0.116050,0.057461


In [53]:
res["error in kg(X reduced)"]=((res["test mse(X reduced)"]+res["train mse(X reduced)"])/20)*std_var
res["error in kg(X)"]=((res["test mse(X)"]+res["train mse(X)"])/20)*std_var

In [54]:
std_var

70772.44640851556

In [55]:
res

,algo,param,test mse(X),train mse(X),test mse(X reduced),train mse(X reduced),error in kg(X reduced),error in kg(X)
0,Linear regression,NaN,0.826416,0.726966,0.172370,0.169471,1209.644910,5496.830813
1,Linear regression with CV,NaN,0.976110,0.700230,0.194807,0.037950,823.639750,5931.934141
2,Elastic Net (tunning with gridsearch),{'lasso__alpha': 0.01},0.770946,0.643378,0.182603,0.179601,1281.702869,5004.756301
3,SGDRegressor,"{loss=squared_epsilon_insensitive,learning_rat...",0.770946,0.643378,0.175284,0.171893,1228.528401,5004.756301
4,SVR,"{kernel='rbf', C=10000, gamma='auto',coef0=1,v...",0.630053,0.092600,0.204866,0.078860,1004.001167,2557.195300
5,RF,{n_estimator=100},0.188616,0.145480,0.130241,0.045007,620.135466,1182.242597
6,XGBoost (tunning with gridsearch),"{'gamma': 0.01, 'learning rate': 0.01, 'max_de...",0.357840,0.148457,0.116050,0.057461,613.989833,1791.591779
